In [9]:
# -*- coding: utf-8 -*-

"""
The :mod:`sklearn.naive_bayes` module implements Naive Bayes algorithms. These
are supervised learning methods based on applying Bayes' theorem with strong
(naive) feature independence assumptions.
"""

# Author: Vincent Michel <vincent.michel@inria.fr>
#         Minor fixes by Fabian Pedregosa
#         Amit Aides <amitibo@tx.technion.ac.il>
#         Yehuda Finkelstein <yehudaf@tx.technion.ac.il>
#         Lars Buitinck
#         Jan Hendrik Metzen <jhm@informatik.uni-bremen.de>
#         (parts based on earlier work by Mathieu Blondel)
#
# License: BSD 3 clause
import warnings

from abc import ABCMeta, abstractmethod

import seaborn as sn
import matplotlib.pyplot as plt
import pandas
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes as nb
import numpy as np
from sklearn.model_selection import train_test_split


import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import label_binarize
from sklearn.utils import check_X_y, check_array, deprecated
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils.fixes import logsumexp
from sklearn.utils.multiclass import _check_partial_fit_first_call
from sklearn.utils.validation import check_is_fitted, check_non_negative, column_or_1d
from sklearn.utils.validation import _check_sample_weight

__all__ = ['BernoulliNB', 'GaussianNB', 'MultinomialNB', 'ComplementNB',
           'CategoricalNB']


class _BaseNB(ClassifierMixin, BaseEstimator, metaclass=ABCMeta):
    """Abstract base class for naive Bayes estimators"""

    @abstractmethod
    def _joint_log_likelihood(self, X):
        """Compute the unnormalized posterior log probability of X

        I.e. ``log P(c) + log P(x|c)`` for all rows x of X, as an array-like of
        shape [n_classes, n_samples].

        Input is passed to _joint_log_likelihood as-is by predict,
        predict_proba and predict_log_proba.
        """

    def _check_X(self, X):
        """To be overridden in subclasses with the actual checks."""
        # Note that this is not marked @abstractmethod as long as the
        # deprecated public alias sklearn.naive_bayes.BayesNB exists
        # (until 0.24) to preserve backward compat for 3rd party projects
        # with existing derived classes.
        return X

    def predict(self, X):
        """
        Perform classification on an array of test vectors X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        C : ndarray of shape (n_samples,)
            Predicted target values for X
        """
        print("I am in the naive bayes predict")
        check_is_fitted(self)
        X = self._check_X(X)
        jll = self._joint_log_likelihood(X)
        print('hmmmmmmmmmmmmmm')
        print(self.classes_[np.argmax(jll, axis=1)])
        return self.classes_[np.argmax(jll, axis=1)]

    def predict_log_proba(self, X):
        check_is_fitted(self)
        X = self._check_X(X)
        jll = self._joint_log_likelihood(X)
        # normalize by P(x) = P(f_1, ..., f_n)
        log_prob_x = logsumexp(jll, axis=1)
        return jll - np.atleast_2d(log_prob_x).T

    def predict_proba(self, X):
        """
        Return probability estimates for the test vector X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        C : array-like of shape (n_samples, n_classes)
            Returns the probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute :term:`classes_`.
        """
        return np.exp(self.predict_log_proba(X))




_ALPHA_MIN = 1e-10


class _BaseDiscreteNB(_BaseNB):
    """Abstract base class for naive Bayes on discrete/categorical data

    Any estimator based on this class should provide:

    __init__
    _joint_log_likelihood(X) as per _BaseNB
    """

    def _check_X(self, X):
        return check_array(X, accept_sparse='csr')

    def _check_X_y(self, X, y):
        return check_X_y(X, y, accept_sparse='csr')

    def _update_class_log_prior(self, class_prior=None):
        n_classes = len(self.classes_)
        print("n_classes")
        
        if class_prior is not None:
            if len(class_prior) != n_classes:
                raise ValueError("Number of priors must match number of"
                                 " classes.")
            self.class_log_prior_ = np.log(class_prior)
            
            
        elif self.fit_prior:
            with warnings.catch_warnings():
                # silence the warning when count is 0 because class was not yet
                # observed
                warnings.simplefilter("ignore", RuntimeWarning)
                log_class_count = np.log(self.class_count_)

                print(self.class_count_)

                print(log_class_count)

                print(np.log(self.class_count_.sum()))


            # empirical prior, with sample_weight taken into account
            self.class_log_prior_ = (log_class_count -
                                     np.log(self.class_count_.sum()))
        else:
            self.class_log_prior_ = np.full(n_classes, -np.log(n_classes))
        print(self.class_log_prior_ )

    def _check_alpha(self):
        if np.min(self.alpha) < 0:
            raise ValueError('Smoothing parameter alpha = %.1e. '
                             'alpha should be > 0.' % np.min(self.alpha))
        if isinstance(self.alpha, np.ndarray):
            if not self.alpha.shape[0] == self.n_features_:
                raise ValueError("alpha should be a scalar or a numpy array "
                                 "with shape [n_features]")
        if np.min(self.alpha) < _ALPHA_MIN:
            warnings.warn('alpha too small will result in numeric errors, '
                          'setting alpha = %.1e' % _ALPHA_MIN)
            return np.maximum(self.alpha, _ALPHA_MIN)
        return self.alpha

    def partial_fit(self, X, y, classes=None, sample_weight=None):
        """Incremental fit on a batch of samples.

        This method is expected to be called several times consecutively
        on different chunks of a dataset so as to implement out-of-core
        or online learning.

        This is especially useful when the whole dataset is too big to fit in
        memory at once.

        This method has some performance overhead hence it is better to call
        partial_fit on chunks of data that are as large as possible
        (as long as fitting in the memory budget) to hide the overhead.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        y : array-like of shape (n_samples,)
            Target values.

        classes : array-like of shape (n_classes) (default=None)
            List of all the classes that can possibly appear in the y vector.

            Must be provided at the first call to partial_fit, can be omitted
            in subsequent calls.

        sample_weight : array-like of shape (n_samples,), default=None
            Weights applied to individual samples (1. for unweighted).

        Returns
        -------
        self : object
        """
        X, y = self._check_X_y(X, y)
        _, n_features = X.shape

        if _check_partial_fit_first_call(self, classes):
            # This is the first call to partial_fit:
            # initialize various cumulative counters
            n_effective_classes = len(classes) if len(classes) > 1 else 2
            self._init_counters(n_effective_classes, n_features)
            self.n_features_ = n_features
        elif n_features != self.n_features_:
            msg = "Number of features %d does not match previous data %d."
            raise ValueError(msg % (n_features, self.n_features_))

        Y = label_binarize(y, classes=self.classes_)
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        if X.shape[0] != Y.shape[0]:
            msg = "X.shape[0]=%d and y.shape[0]=%d are incompatible."
            raise ValueError(msg % (X.shape[0], y.shape[0]))

        # label_binarize() returns arrays with dtype=np.int64.
        # We convert it to np.float64 to support sample_weight consistently
        Y = Y.astype(np.float64, copy=False)
        if sample_weight is not None:
            sample_weight = np.atleast_2d(sample_weight)
            Y *= check_array(sample_weight).T

        class_prior = self.class_prior

        # Count raw events from data before updating the class log prior
        # and feature log probas
        self._count(X, Y)

        # XXX: OPTIM: we could introduce a public finalization method to
        # be called by the user explicitly just once after several consecutive
        # calls to partial_fit and prior any call to predict[_[log_]proba]
        # to avoid computing the smooth log probas at each call to partial fit
        alpha = self._check_alpha()
        self._update_feature_log_prob(alpha)
        self._update_class_log_prior(class_prior=class_prior)
        return self

    def fit(self, X, y, sample_weight=None):
        """Fit Naive Bayes classifier according to X, y

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        y : array-like of shape (n_samples,)
            Target values.

        sample_weight : array-like of shape (n_samples,), default=None
            Weights applied to individual samples (1. for unweighted).

        Returns
        -------
        self : object
        """
        X, y = self._check_X_y(X, y)
        _, n_features = X.shape
        self.n_features_ = n_features

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        print("fit")
        print(Y.shape)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)
            print("comcaascdk")
            print(Y.T)


        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # We convert it to np.float64 to support sample_weight consistently;
        # this means we also don't have to cast X to floating point
        if sample_weight is not None:
            Y = Y.astype(np.float64, copy=False)
            sample_weight = np.asarray(sample_weight)
            sample_weight = np.atleast_2d(sample_weight)
            Y *= check_array(sample_weight).T

        class_prior = self.class_prior

        # Count raw events from data before updating the class log prior
        # and feature log probas
        n_effective_classes = Y.shape[1]

        self._init_counters(n_effective_classes, n_features)
        self._count(X, Y)
        alpha = self._check_alpha()
        self._update_feature_log_prob(alpha)
        self._update_class_log_prior(class_prior=class_prior)
        return self

    def _init_counters(self, n_effective_classes, n_features):
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.feature_count_ = np.zeros((n_effective_classes, n_features),
                                       dtype=np.float64)

    # XXX The following is a stopgap measure; we need to set the dimensions
    # of class_log_prior_ and feature_log_prob_ correctly.
    def _get_coef(self):
        return (self.feature_log_prob_[1:]
                if len(self.classes_) == 2 else self.feature_log_prob_)

    def _get_intercept(self):
        return (self.class_log_prior_[1:]
                if len(self.classes_) == 2 else self.class_log_prior_)

    coef_ = property(_get_coef)
    intercept_ = property(_get_intercept)

    def _more_tags(self):
        return {'poor_score': True}



class ComplementNB(_BaseDiscreteNB):
    def __init__(self, alpha=1.0, fit_prior=True, class_prior=None,
                 norm=False):
        self.alpha = alpha
        self.fit_prior = fit_prior
        self.class_prior = class_prior
        self.norm = norm

    def _more_tags(self):
        return {'requires_positive_X': True}

    def _count(self, X, Y):
        """Count feature occurrences."""
        print("I am in the counter")
        print(Y.T)
        print(Y.T.shape)
        check_non_negative(X, "ComplementNB (input X)")
        self.feature_count_ += safe_sparse_dot(Y.T, X)
        self.class_count_ += Y.sum(axis=0)
        self.feature_all_ = self.feature_count_.sum(axis=0)
        
        print(self.feature_count_)
        print(self.feature_all_)

    def _update_feature_log_prob(self, alpha):
        """Apply smoothing to raw counts and compute the weights."""
        alpha = 1
        comp_count = self.feature_all_ + alpha - self.feature_count_
        print("-------------------comp_cpunt-------------")
        print(comp_count)
        print(comp_count.sum(axis=1, keepdims=True))
        logged = np.log(comp_count / comp_count.sum(axis=1, keepdims=True))
        print("----------loggged------")
        print(logged)
        # _BaseNB.predict uses argmax, but ComplementNB operates with argmin.
        if self.norm:
            summed = logged.sum(axis=1, keepdims=True)
            feature_log_prob = logged / summed
        else:
            feature_log_prob = -logged
            print("-------feature----------")
            print(feature_log_prob)
        self.feature_log_prob_ = feature_log_prob

    def _joint_log_likelihood(self, X):
        """Calculate the class scores for the samples in X."""
        print("x")
        print(X)
        jll = safe_sparse_dot(X, self.feature_log_prob_.T)
        print("....jlllll........")
        print(jll)
        if len(self.classes_) == 1:
            jll += self.class_log_prior_
            print(jll)
        return jll





dataframe = pandas.read_table('/content/mixed200.txt')
data = dataframe.drop("No.", axis=1)
data = data.drop("StdPageRank", axis=1)
data = data.drop("VarPageRank", axis=1)
dataset = data.values



# split into input (X) and output (Y) variables
training = 57748
testing = 24749

l = 82497
X = dataset[:,0:]
# print(X.shape)
labels = []
for i in range (0,l):
  if X[i][9] == 1:
    labels.append(1)
  else:
    labels.append(0)

features = X[0:l,0:9]




# Split our data
train, test, train_labels, test_labels = features[0:training],features[training:testing+training],labels[0:training],labels[training:testing+training]

  # Initialize our classifier



gnb = ComplementNB()




# Train our classifier
print(len(train_labels))
model =gnb.fit(train,train_labels)
preds = gnb.predict(test)
print("----------------------------CNB--------------------------")
print(accuracy_score(test_labels, preds))






57748
fit
(57748, 1)
comcaascdk
[[1 0 0 ... 1 1 0]
 [0 1 1 ... 0 0 1]]
I am in the counter
[[1 0 0 ... 1 1 0]
 [0 1 1 ... 0 0 1]]
(2, 57748)
[[7.20184000e+05 1.39657100e+06 2.40487000e+05 1.47190000e+04
  2.34285000e+05 1.47360000e+04 4.03109598e+02 3.55455770e+03
  2.21042790e+02]
 [1.03530300e+06 1.82424200e+06 1.71753000e+05 4.10530000e+04
  1.67663000e+05 4.10510000e+04 1.76403744e+03 2.66697401e+03
  6.30614075e+02]]
[1.75548700e+06 3.22081300e+06 4.12240000e+05 5.57720000e+04
 4.01948000e+05 5.57870000e+04 2.16714703e+03 6.22153171e+03
 8.51656865e+02]
-------------------comp_cpunt-------------
[[1.03530400e+06 1.82424300e+06 1.71754000e+05 4.10540000e+04
  1.67664000e+05 4.10520000e+04 1.76503744e+03 2.66797401e+03
  6.31614075e+02]
 [7.20185000e+05 1.39657200e+06 2.40488000e+05 1.47200000e+04
  2.34286000e+05 1.47370000e+04 4.04109598e+02 3.55555770e+03
  2.22042790e+02]]
[[3286135.62551802]
 [2625169.71008624]]
----------loggged------
[[-1.15501719 -0.58854719 -2.95140435 -4.3

57748
fit
(57748, 1)
comcaascdk
[[1 0 0 ... 1 1 0]
 [0 1 1 ... 0 0 1]]
I am in the counter
[[1 0 0 ... 1 1 0]
 [0 1 1 ... 0 0 1]]
(2, 57748)
[[7.20184000e+05 1.39657100e+06 2.40487000e+05 1.47190000e+04
  2.34285000e+05 1.47360000e+04 4.03109598e+02 3.55455770e+03
  2.21042790e+02]
 [1.03530300e+06 1.82424200e+06 1.71753000e+05 4.10530000e+04
  1.67663000e+05 4.10510000e+04 1.76403744e+03 2.66697401e+03
  6.30614075e+02]]
[1.75548700e+06 3.22081300e+06 4.12240000e+05 5.57720000e+04
 4.01948000e+05 5.57870000e+04 2.16714703e+03 6.22153171e+03
 8.51656865e+02]
-------------------comp_cpunt-------------
[[1.03530400e+06 1.82424300e+06 1.71754000e+05 4.10540000e+04
  1.67664000e+05 4.10520000e+04 1.76503744e+03 2.66797401e+03
  6.31614075e+02]
 [7.20185000e+05 1.39657200e+06 2.40488000e+05 1.47200000e+04
  2.34286000e+05 1.47370000e+04 4.04109598e+02 3.55555770e+03
  2.22042790e+02]]
[[3286135.62551802]
 [2625169.71008624]]
----------loggged------
[[-1.15501719 -0.58854719 -2.95140435 -4.3

In [5]:
import seaborn as sn
import matplotlib.pyplot as plt
import pandas
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes as nb
import numpy as np
from sklearn.model_selection import train_test_split


dataframe = pandas.read_table('/content/mixed200.txt')
data = dataframe.drop("No.", axis=1)
data = data.drop("StdPageRank", axis=1)
data = data.drop("VarPageRank", axis=1)
dataset = data.values



# split into input (X) and output (Y) variables
training = 57748
testing = 24749

l = 82497
X = dataset[:,0:]
# print(X.shape)
labels = []
for i in range (0,l):
  if X[i][9] == 1:
    labels.append(1)
  else:
    labels.append(0)

features = X[0:l,0:9]




# Split our data
train, test, train_labels, test_labels = features[0:training],features[training:testing+training],labels[0:training],labels[training:testing+training]

  # Initialize our classifier



gnb = nb.ComplementNB()




# Train our classifier
print(len(train_labels))
model =gnb.fit(train,train_labels)
preds = gnb.predict(test)
print("----------------------------CNB--------------------------")
print(accuracy_score(test_labels, preds))




57748
----------------------------CNB--------------------------
0.9053295082629601
